## 재귀를 반복문으로 구현

In [12]:
class Variable:
    
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)
                

In [17]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    

class Square(Function):
    def forward(self, input):
        x = input **2
        return x
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

    


In [18]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0) # 자기자신을 미분한게 1이기 때문에
y.backward()
print(x.grad)

3.297442541400256


# 함수로 더편리하게 바꾸기

In [19]:
def square(x):
    return Square()(x)
def exp(x):
    return Exp()(x)

In [20]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [25]:
# np.ones_like를 사용하게 되면 배열을 전부 1로 치환 해줌
print(np.ones_like(8))

print(np.ones_like([133,6,7]))

1
[1 1 1]


In [26]:
# 항상 치던 y.grad = np.array(1.0)을  안치도록
class Variable:
    
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

In [27]:
# 다시 테스트
x = Variable(0.5)
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


In [28]:
# ndarray 만 취급하기
class Variable:
    
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} 는 지원하지 않습니다.')
        self.data = data
        self.grad = None
        self.creator = None
        

In [30]:
x = Variable(np.array(1.0))
x  = Variable(None)


In [31]:
x = Variable(1)

TypeError: <class 'int'> 는 지원하지 않습니다.

- 넘파이에 배열을 넣고 조작하면 output도 ndarray 이다


In [35]:
x = np.array([1,1,1])
print(type(x))
y = x**2
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


- 하지만, 배열이 0 차원일 때는 문제가 있다  ndarray 가 np.float 64 나 32 로 데이터 타입이 변하게 된다.


In [36]:
x = np.array(1.0)
y = x **2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [37]:
def as_array(x):
    if np.isscalar(x):
        return np.array((x))
    return x

In [39]:
np.isscalar(np.float64(1.0)), np.isscalar(np.array([1,2,4]))

(True, False)

In [40]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input= input
        self.output = output
        return output